In [ ]:
import sklearn
import numpy as np
import os
import time
import random
from tqdm import tqdm
import pickle
from sklearn.decomposition import PCA

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Preprocessing():
  def __init__(self, pca_n_components):
    self.pca = PCA(n_components = pca_n_components)

  def kd_tree(self, points, k):
    points = points[points[:, k-1].argsort()]
    k_max = len(points[0])
    l = len(points)
    if l==1:
      return [points[0]]
    elif l == 2:
      return [points[0], points[1]]
    else:
      m = int((len(points)-1)/2)
      sorted_points_left = self.kd_tree(points[:m], (k+1)%k_max)
      sorted_points_right = self.kd_tree(points[m+1:], (k+1)%k_max)
      sorted_points = [points[m]] + sorted_points_left+ sorted_points_right
      return sorted_points

  def get_data(self, folder_path = '/content/drive/MyDrive/preimage_task/shapenet-chairs-pcd'):
    file_list = [os.path.join(folder_path, x) for x in os.listdir(folder_path)]
    point_clouds = []
    sorted_point_clouds = []
    for file_path in tqdm(file_list, ncols = 100):
        f = open(file_path)
        points = f.readlines()[10:]
        points = np.array([[float(x.split(' ')[0]), float(x.split(' ')[1]), float(x.split(' ')[2][:-2])] for x in points])
        sorted_points = self.kd_tree(points, 1)
        point_clouds.append(points)
        sorted_point_clouds.append(np.array(sorted_points))
    point_clouds = np.array(point_clouds)
    sorted_point_clouds = np.array(sorted_point_clouds)
    sorted_point_clouds_2d = sorted_point_clouds.reshape((-1, 3000))
    print(f"Size of point_cloud : {point_clouds.shape}\nSize of sorted kd_tree point cloud : {sorted_point_clouds.shape}\nSize of sorted 2d kd_tree point cloud : {sorted_point_clouds_2d.shape}")
    return point_clouds, sorted_point_clouds, sorted_point_clouds_2d

  def pca_basis_space(self, data):
    self.pca.fit(data)
    return self.pca.components_

  def pca_projections(self, data):
    return self.pca.fit_transform(data)
  
  def reconstruction_error(self, swapped_col, sub_uTu, full_mat_mul, swapped):
    swapped_mean = np.mean(swapped_col, axis = 0)
    matmul_error = np.matmul((swapped_col - swapped_mean), sub_uTu)
    mat_mul = full_mat_mul + matmul_error
    loss = np.sum(np.square(mat_mul - swapped + np.mean(swapped, axis = 0)), axis = 1)
    return loss, matmul_error, mat_mul

  def optimizing_point_ordering(self, sorted_point_clouds_2d, outer_loop_limit = 1000, inner_loop_limit = 10000, save_path = None):
    corrected = sorted_point_clouds_2d.copy()
    sample = {x: [3*x, 3*x+1, 3*x+2] for x in range(0, 1000)}
    u = self.pca_basis_space(corrected)

    for i in range(outer_loop_limit):
      mean = np.mean(corrected, axis = 0)
      uTu = np.matmul(u.transpose(), u)
      full_mat_mul = np.matmul(corrected - mean, uTu)
      min_loss = np.sum(np.square(full_mat_mul + mean  - corrected), axis = 1)
      print(f'iteration = {i}')
      for _ in tqdm(range(inner_loop_limit), ncols = 100):
        idx = np.array([random.sample(sample.keys(), 2)]).reshape((2,))
        idx_6 =  np.append(sample[idx[0]], sample[idx[1]])
        idx_6_rev = np.append(sample[idx[1]], sample[idx[0]])

        swapped_col = corrected[:, idx_6_rev]
        unswapped_col = corrected[:, idx_6]
        swapped = corrected.copy()
        swapped[:, idx_6] = swapped_col
        
        sub_uTu = uTu[idx_6, :] - uTu[idx_6_rev, :]
        loss, matmul_error, mat_mul = self.reconstruction_error(swapped_col, sub_uTu, full_mat_mul, swapped)
        
        c1 = loss>=min_loss
        c2 = loss<min_loss
        min_loss = min_loss*c1  + loss*c2
        c1 = c1.reshape((-1, 1))
        c2 = c2.reshape((-1, 1))
        final_col = unswapped_col*c1 + swapped_col*c2
        corrected[:, idx_6] = final_col
        full_mat_mul = full_mat_mul*c1  + mat_mul*c2

      self.pca.fit(corrected)
      u = self.pca_basis_space(sorted_point_clouds_2d)
      if save_path is not None:
        np.save(save_path, corrected)

    return corrected

In [ ]:
preprocess = Preprocessing(pca_n_components= 100)
point_clouds, sorted_point_clouds, sorted_point_clouds_2d = preprocess.get_data(folder_path = '/content/drive/MyDrive/preimage_task/shapenet-chairs-pcd')

100%|███████████████████████████████████████████████████████████| 5000/5000 [00:54<00:00, 91.84it/s]


Size of point_cloud : (5000, 1000, 3)
Size of sorted kd_tree point cloud : (5000, 1000, 3)
Size of sorted 2d kd_tree point cloud : (5000, 3000)


In [ ]:
corrected = preprocess.optimizing_point_ordering(sorted_point_clouds_2d)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
criterion = nn.BCELoss()

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        *[nn.Linear(100, 100, bias = True), nn.BatchNorm1d(100), nn.ReLU(inplace=True)]*4, 
        nn.Linear(100, 100, bias = True),
        nn.BatchNorm1d(100),
        nn.Tanh()
        )
  
  def forward(self, x):
    return self.layers(x)

class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Sequential(nn.Linear(100, 100, bias = True), nn.BatchNorm1d(100), nn.LeakyReLU(0.2,inplace=True))
    self.layer2 = nn.Sequential(nn.Linear(100, 100, bias = True), nn.BatchNorm1d(100), nn.LeakyReLU(0.2,inplace=True))
    self.layer3 = nn.Sequential(nn.Linear(100, 100, bias = True), nn.BatchNorm1d(100), nn.LeakyReLU(0.2,inplace=True))
    self.layer4 = nn.Sequential(nn.Linear(100, 100, bias = True), nn.BatchNorm1d(100), nn.LeakyReLU(0.2,inplace=True))
    self.layer5 = nn.Sequential(nn.Linear(100, 1, bias = True), nn.Sigmoid()) 
  
  def forward(self, x):
    out1 = self.layer1(x)
    out2 = self.layer2(out1)
    out3 = self.layer3(out2)
    out4 = self.layer4(out3)
    out5 = self.layer5(out4)
    return out5, out4, out3, out2, out1

def init_weights(l):
    if isinstance(l, nn.Linear):
      nn.init.normal_(l.weight, 0, 0.02)
    elif  isinstance(l, nn.BatchNorm1d):
      nn.init.normal_(l.weight.data, 1.0, 0.02)
      nn.init.constant_(l.bias.data, 0)


def discriminator_loss(pred, target):
    return criterion(pred, target)

def generator_loss(t_activations, z_activations):
    loss = 0
    for t_act, z_act in zip(t_activations, z_activations):
      loss += (torch.linalg.norm(torch.mean(t_act, 0) - torch.mean(z_act, 0), 2)**2 + torch.linalg.norm(torch.cov(t_act.transpose(0,1)) - torch.cov(z_act.transpose(0,1)), 2)**2)
    return loss


def fake_point_cloud_generator(data, number_of_fake_points = 1000 ,checkpoint_path = None, generator = None):
  if checkpoint_path is not None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load(checkpoint_path, map_location = device)
    gen = Generator().to(device)
    gen.load_state_dict(checkpoint['gen_state_dict'])
  else:
    gen = generator

  pca = PCA(n_components = 100)
  pca.fit(data.reshape((-1, 3000)))
  noise = torch.randn((number_of_fake_points,100))*2 - 1
  fake_coeff = gen(noise)
  pc_fake = pca.inverse_transform(fake_coeff.detach().numpy())
  pc_fake = pc_fake.reshape((-1, 1000, 3))

  return pc_fake

def Training(dataset, save_name = None, num_epochs = 1000, checkpoint_path = None):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    pca = PCA(n_components= 100)
    dataset = dataset.reshape((-1, 3000))
    data = pca.fit_transform(dataset)
    data = torch.from_numpy(data).float().to(device)
    
    gen = Generator().to(device)
    dis = Discriminator().to(device)
    optimizer_g = torch.optim.Adam(gen.parameters(), lr = 0.0025)
    optimizer_d = torch.optim.Adam(dis.parameters(), lr = 0.0001)

    if checkpoint_path == None:
      epoch = 1
      gen.apply(init_weights)
      dis.apply(init_weights)
      g_loss = [] 
      d_loss = [] 
      d_acc = [] 
      train_discriminator = True

    else:
      checkpoint = torch.load(checkpoint_path, map_location = torch.device(device))
      epoch = checkpoint['epoch'] + 1
      gen.load_state_dict(checkpoint['gen_state_dict'])
      dis.load_state_dict(checkpoint['dis_state_dict'])
      optimizer_g.load_state_dict(checkpoint['optimizer_g_state_dict'])
      optimizer_d.load_state_dict(checkpoint['optimizer_d_state_dict'])
      g_loss = checkpoint['g_loss']
      d_loss = checkpoint['d_loss']
      d_acc = checkpoint['d_acc']
      train_discriminator = checkpoint['train_discriminator']
  

    while(epoch < num_epochs + 1):

      if train_discriminator:
          #Training discriminator
          dis.zero_grad()
          real_out = dis(data)
          real_label = torch.full_like(real_out[0], 1, device= device)
          real_loss = criterion(real_out[0], real_label)
          noise = torch.randn(data.shape, device = device)*2 - 1
          fake_coeff = gen(noise)
          fake_out = dis(fake_coeff)
          fake_label = torch.full_like(real_label, 0, device = device)
          fake_loss = criterion(fake_out[0], fake_label)
          dis_acc = (torch.sum(fake_out[0] < 0.5) + torch.sum(real_out[0]  > 0.5))*50/real_out[0].shape[0]
          dis_loss = real_loss + fake_loss
          dis_loss.backward()
          optimizer_d.step()


      #Trainig generator
      gen.zero_grad()
      real_out = dis(data)
      noise = torch.randn(data.shape, device = device)*2 - 1
      fake_coeff = gen(noise)
      fake_out = dis(fake_coeff)
      gen_loss = generator_loss(real_out[1:], fake_out[1:])
      gen_loss.backward()
      optimizer_g.step()
    
      g_loss.append(gen_loss) 
      if train_discriminator:
          d_loss.append(dis_loss)
          d_acc.append(dis_acc)

      print("===> Epoch({}/{}): G_Loss: {:.4f} ".format(
        epoch, num_epochs, gen_loss.item()), end = "")
      if train_discriminator:
          print("D_Loss: {:.4f} D_Accuracy: {:.4f}".format(dis_loss.item(), dis_acc.item()), end = "")
      print("")

      if save_name is not None:
        torch.save({
            'epoch' : epoch,
            'num_epochs': num_epochs,
            'gen_state_dict' : gen.state_dict(),
            'dis_state_dict' : dis.state_dict(),
            'optimizer_g_state_dict' : optimizer_g.state_dict(),
            'optimizer_d_state_dict' : optimizer_d.state_dict(),
            'g_loss' : g_loss,
            'd_loss' : d_loss,
            'd_acc' : d_acc,
            'train_discriminator' : train_discriminator
        }, "/content/drive/MyDrive/preimage_task/gans_tanh_init_wt_crr_iter_0/" + save_name + ".pth" )
      
      if torch.mean(torch.tensor(d_acc[-40:])).item() > 90:
        train_discriminator = False

      epoch+=1

    return gen, d_loss, g_loss, d_acc

In [ ]:
loss = y*np.log10(z) + (1-y)*np.log10((1-z))

In [ ]:
gen, d_loss, g_loss, d_acc = Training(corrected, num_epochs = 2500)

===> Epoch(1/2500): G_Loss: 155.4699 D_Loss: 1.3932 D_Accuracy: 47.3800
===> Epoch(2/2500): G_Loss: 97.8572 D_Loss: 1.3909 D_Accuracy: 49.2700
===> Epoch(3/2500): G_Loss: 88.0798 D_Loss: 1.3892 D_Accuracy: 49.5900
===> Epoch(4/2500): G_Loss: 63.5626 D_Loss: 1.3883 D_Accuracy: 50.1700
===> Epoch(5/2500): G_Loss: 65.9803 D_Loss: 1.3867 D_Accuracy: 50.9300
===> Epoch(6/2500): G_Loss: 64.8966 D_Loss: 1.3854 D_Accuracy: 51.3400
===> Epoch(7/2500): G_Loss: 57.7458 D_Loss: 1.3842 D_Accuracy: 52.5600
===> Epoch(8/2500): G_Loss: 48.5564 D_Loss: 1.3832 D_Accuracy: 52.8000
===> Epoch(9/2500): G_Loss: 44.3228 D_Loss: 1.3823 D_Accuracy: 53.8300
===> Epoch(10/2500): G_Loss: 44.6065 D_Loss: 1.3813 D_Accuracy: 54.0600
===> Epoch(11/2500): G_Loss: 37.1304 D_Loss: 1.3805 D_Accuracy: 54.6900
===> Epoch(12/2500): G_Loss: 37.5032 D_Loss: 1.3798 D_Accuracy: 54.8300
===> Epoch(13/2500): G_Loss: 39.9722 D_Loss: 1.3790 D_Accuracy: 55.0400
===> Epoch(14/2500): G_Loss: 36.1717 D_Loss: 1.3782 D_Accuracy: 55.5000
